In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 1. Load data
parcels   = pd.read_csv('/content/merged_property_data.csv')
leases    = pd.read_csv('/content/Leases.csv')
occupancy = pd.read_csv('/content/Major Market Occupancy Data-revised.csv')

# 2. Parcel → RBA
parcel_small = parcels[['costarID','Net Rent Area']].rename(columns={'Net Rent Area':'RBA'})

# 3. Merge and clean
pl = leases.merge(parcel_small, on='costarID', how='left')
if 'RBA_x' in pl: pl.drop(columns=['RBA_x'], inplace=True)
if 'RBA_y' in pl: pl.rename(columns={'RBA_y':'RBA'}, inplace=True)

# 4. Weighted availability per market‑quarter
lease_agg = (
    pl
    .groupby(['market','year','quarter'], as_index=False)
    .apply(lambda df: pd.Series({
        'market_avail_prop': (df['availability_proportion'] * df['RBA']).sum() / df['RBA'].sum(),
        'total_RBA': df['RBA'].sum(),
        'bldg_count': df['costarID'].nunique()
    }))
    .reset_index(drop=True)
)

# 5. Merge with occupancy
markets_q = occupancy[['market','year','quarter']].drop_duplicates()
full = (
    markets_q
    .merge(lease_agg, on=['market','year','quarter'], how='left')
    .merge(occupancy[['market','year','quarter','avg_occupancy_proportion']],
           on=['market','year','quarter'], how='left')
    .rename(columns={'avg_occupancy_proportion':'occ_prop'})
)
full[['market_avail_prop','total_RBA','bldg_count']] = full[
    ['market_avail_prop','total_RBA','bldg_count']
].fillna(0)

# 6. Composite score
full['norm_avail'] = full['market_avail_prop']
full['norm_occ']   = full['occ_prop']
full['norm_scale'] = full['total_RBA']/full['total_RBA'].max()
full['norm_count'] = full['bldg_count']/full['bldg_count'].max()
w1,w2,w3,w4 = 0.3,0.3,0.2,0.2
full['score'] = w1*full['norm_avail'] + w2*full['norm_occ'] + w3*full['norm_scale'] + w4*full['norm_count']

# 7. Filter to 2024 and drop zero‑RBA
recent = full[(full['year']==2024)&(full['total_RBA']>0)].copy()

# 8. Plot **all** markets in 2024
avail_cut, occ_cut = 20, 50
fig, ax = plt.subplots(figsize=(12,7))

# Quadrant shading
ax.axvspan(avail_cut,100,occ_cut,100, facecolor='#D0F0C0', alpha=0.3)
ax.axvspan(avail_cut,100,0,occ_cut,     facecolor='#FEE0C0', alpha=0.3)
ax.axvspan(0,avail_cut,occ_cut,100,     facecolor='#C0D0FE', alpha=0.3)
ax.axvspan(0,avail_cut,0,occ_cut,       facecolor='#F4C0E0', alpha=0.3)

# Sizes & scatter
min_s, max_s = 50, 2000
sizes = np.interp(recent['total_RBA'], (recent['total_RBA'].min(), recent['total_RBA'].max()), (min_s, max_s))
sc = ax.scatter(
    recent['market_avail_prop']*100,
    recent['occ_prop']*100,
    s=sizes,
    c=recent['score'],
    cmap='viridis',
    alpha=0.8,
    edgecolors='k', linewidth=0.3
)

# Threshold lines
ax.axvline(avail_cut, color='k', linestyle='--')
ax.axhline(occ_cut,   color='k', linestyle='--')

# Market annotations
for _, r in recent.iterrows():
    ax.annotate(
        r['market'],
        (r['market_avail_prop']*100, r['occ_prop']*100),
        textcoords='offset points', xytext=(3,3), ha='left', fontsize=7
    )

# Quadrant labels
ax.text(0.75,0.75,'Sweet Spot', transform=ax.transAxes, ha='center', va='center', color='#006400', fontsize=14, alpha=0.6)
ax.text(0.75,0.25,'Oversupply', transform=ax.transAxes, ha='center', va='center', color='#8B4513', fontsize=14, alpha=0.6)
ax.text(0.25,0.75,'Tight',      transform=ax.transAxes, ha='center', va='center', color='#00008B', fontsize=14, alpha=0.6)
ax.text(0.25,0.25,'Stagnant',   transform=ax.transAxes, ha='center', va='center', color='#8B008B', fontsize=14, alpha=0.6)

# Colorbar & labels
cbar = fig.colorbar(sc, ax=ax)
cbar.set_label('Composite Score (%)')
ax.set_xlabel('Availability (%)')
ax.set_ylabel('Occupancy (%)')
ax.set_title('Class A Markets by Sweet‑Spot Score in 2024')
ax.set_xlim(0, recent['market_avail_prop'].max()*100 + 5)
ax.set_ylim(recent['occ_prop'].min()*100 - 5, 100)
ax.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()
